In [ ]:
%pip install tensorflow-gpu

In [ ]:
import os
import tensorflow as tf
import numpy as np
import math
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
def build_pinn(input_shape=(64, 64, 64, 4)):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Dense(64, activation='tanh')(inputs)
    x = layers.Dense(64, activation='tanh')(x)
    x = layers.Dense(64, activation='tanh')(x)
    
    outputs = layers.Dense(1, activation=None)(x)  # No activation for PDE solution
    
    return Model(inputs, outputs)

In [ ]:
def compute_physics_residual(x, y_pred):
    """Compute the residual of the PDE for fire spread"""
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x)  # Watch the input tensor x to compute gradients with respect to it
        tape.watch(y_pred)  # Watch y_pred (fire intensity) for derivatives

        # Assuming y_pred is fire intensity, compute the first derivative (gradient)
        grad = tape.gradient(y_pred, x)  # Gradient of y_pred w.r.t. input x (spatial derivatives)
        laplacian = tf.reduce_sum(grad**2, axis=-1)  # Approximate Laplacian (second spatial derivative)

    # Compute reaction term (you can adjust this based on your PDE)
    reaction_term = y_pred * (1 - y_pred)  # Example nonlinear reaction term

    # Combine them to get the PDE residual
    residual = laplacian + reaction_term  # For example, a simple reaction-diffusion model

    return residual

In [ ]:
def pinn_loss(y_true, y_pred, residual):
    data_loss = tf.keras.losses.MSE(y_true, y_pred)  # Mean squared error on labeled data
    physics_loss = tf.reduce_mean(tf.square(residual))  # Physics constraint (PDE residual)
    return data_loss + physics_loss
    

In [ ]:
def load_tiff_image(file_path):
    """Load a .tiff image and normalize it."""
    img = tf.io.read_file(file_path)
    img = tf.image.decode_tiff(img, channels=4)  # Assuming 4 channels (e.g., fire, wind, terrain, etc.)
    img = tf.image.resize(img, [64, 64])  # Resize to the expected input shape
    img = img / 255.0  # Normalize to [0, 1]
    return img

In [ ]:
# Example of loading all .tiff files in a directory
def load_data_from_directory(directory):
    """Load .tiff files into a dataset."""
    file_paths = [os.path.join(directory, fname) for fname in os.listdir(directory) if fname.endswith('.tiff')]
    images = [load_tiff_image(f) for f in file_paths]
    return tf.data.Dataset.from_tensor_slices(np.array(images))

In [ ]:
# Load and prepare the dataset
train_dataset = load_data_from_directory('/data/ai_club/fire/WildfireSpreadLS/2018/fire_21458798')
train_dataset = train_dataset.batch(2).repeat()  # Batch and repeat for training

In [ ]:
# Cosine annealing learning rate scheduler
def cosine_annealing(epoch, lr, T_max=50, eta_min=1e-6):
    return eta_min + (lr - eta_min) * (1 + math.cos(math.pi * epoch / T_max)) / 2

lr_scheduler = LearningRateScheduler(lambda epoch, lr: cosine_annealing(epoch, lr, 50))

# Set up the optimizer and custom training loop
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

@tf.function
def train_step(x_train, y_train, model):
    with tf.GradientTape() as tape:
        y_pred = model(x_train, training=True)
        # Compute physics residual here (e.g., fire propagation PDE)
        physics_residual = compute_physics_residual(x_train, y_pred)
        loss = pinn_loss(y_train, y_pred, physics_residual)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Placeholder training loop
for epoch in range(10):
    for x_batch, y_batch in train_dataset:
        loss = train_step(x_batch, y_batch, model)
        print(f'Epoch {epoch}, Loss: {loss}')